In [2]:
import os
import re

from dotenv import load_dotenv
from langchain import hub
from langchain.agents import AgentType, load_tools
from langchain.agents.initialize import initialize_agent
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import JSONAgentOutputParser
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
# from langchain.chat_models import ChatOpenAI
# from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI
from langchain.tools import ArxivQueryRun, WikipediaQueryRun
from langchain.tools.render import render_text_description_and_args
from langchain.utilities import ArxivAPIWrapper, WikipediaAPIWrapper
from langchain.prompts import MessagesPlaceholder
# from langchain.schema import ChatMessage, SystemMessage
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.base import BaseCallbackManager

from llamp.mp.agents import (
    MPSummaryExpert,
    MPThermoExpert,
    MPElasticityExpert,
    MPDielectricExpert,
    MPPiezoelectricExpert,
    MPMagnetismExpert,
    MPElectronicExpert,
    MPSynthesisExpert,
    MPStructureRetriever,
)
# from llamp.arxiv.agents import ArxivAgent

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", None)
OPENAI_ORGANIZATION = os.getenv("OPENAI_ORGANIZATION", None)

OPENAI_GPT_MODEL = "gpt-4-1106-preview"
# OPENAI_GPT_MODEL = "gpt-4-0125-preview"
# OPENAI_GPT_MODEL = "gpt-3.5-turbo-1106"
# OPENAI_GPT_MODEL = "gpt-4"

No module named 'phonopy'
No module named 'phonopy'


In [4]:
top_llm = ChatOpenAI(
    temperature=0,
    model=OPENAI_GPT_MODEL,
    openai_api_key=OPENAI_API_KEY,
    openai_organization=OPENAI_ORGANIZATION,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

bottom_callback_handler = StreamingStdOutCallbackHandler()

bottom_llm = ChatOpenAI(
    temperature=0,
    model=OPENAI_GPT_MODEL,
    openai_api_key=OPENAI_API_KEY,
    openai_organization=OPENAI_ORGANIZATION,
    max_retries=5,
    streaming=True,
    callbacks=[bottom_callback_handler],
)


wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
arxiv = ArxivQueryRun(api_wrapper=ArxivAPIWrapper())

tools = [
    MPThermoExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPElasticityExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPDielectricExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPMagnetismExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPElectronicExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPPiezoelectricExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPSummaryExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPSynthesisExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPStructureRetriever(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    # ArxivAgent(llm=bottom_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    arxiv,
    wikipedia,
]
tools += load_tools(["llm-math"], llm=bottom_llm)

prompt = hub.pull("hwchase17/react-multi-input-json")
prompt.messages[0].prompt.template = (
    re.sub(
        r"\s+",
        " ",
        """You are a data-aware agent that can consult materials-related
    data through Materials Project (MP) database, arXiv, and Wikipedia. Ask 
    user to clarify their queries if needed. Please note that you don't have 
    direct control over MP but through multiple assistant agents to help you. 
    You need to provide complete context in the input for them to do their job.
    """,
    ).replace("\n", " ")
    + prompt.messages[0].prompt.template
)

prompt = prompt.partial(
    tools=render_text_description_and_args(tools),
    tool_names=", ".join([t.name for t in tools]),
)

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
    }
    | prompt
    | top_llm.bind(stop=["Observation"])
    # | map_reduce_chain  # TODO: Add map-reduce after LLM
    | JSONAgentOutputParser()
)

conversational_memory = ConversationBufferWindowMemory(
    memory_key="chat_history", k=5, return_messages=True
)

agent_kwargs = {
    "handle_parsing_errors": True,
    "extra_prompt_messages": [
        MessagesPlaceholder(variable_name="chat_history"),
    ],
    "early_stopping_method": "generate",
}

agent_executor = initialize_agent(
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools,
    llm=top_llm,
    verbose=True,
    max_iterations=20,
    memory=conversational_memory,
    agent_kwargs=agent_kwargs,
    handle_parsing_errors=True,
    callback_manager=BaseCallbackManager(handlers=[bottom_callback_handler]),
)

In [3]:
agent_executor.invoke(
    {
        "input": "Please give me the mass density (in g/cm^3) and volume per atom (in Angstrom^3/atom) of stable NaCl.",
    }
)



> Entering new AgentExecutor chain...
Action:
```json
{
  "action": "MPThermoExpert",
  "action_input": {
    "input": "What is the mass density and volume per atom of stable NaCl?"
  }
}
```Action:
```json
{
  "action": "MPThermoExpert",
  "action_input": {
    "input": "What is the mass density and volume per atom of stable NaCl?"
  }
}
```

> Entering new AgentExecutor chain...
Thought: To find the mass density and volume per atom of stable sodium chloride (NaCl), I will use the search_materials_thermo__get tool with the appropriate filters.

Action:
```
{
  "action": "search_materials_thermo__get",
  "action_input": {
    "formula": "NaCl",
    "is_stable": true,
    "fields": "density,volume,nsites"
  }
}
```Thought: To find the mass density and volume per atom of stable sodium chloride (NaCl), I will use the search_materials_thermo__get tool with the appropriate filters.

Action:
```
{
  "action": "search_materials_thermo__get",
  "action_input": {
    "formula": "NaCl",
    "i

Retrieving ThermoDoc documents:   0%|          | 0/3 [00:00<?, ?it/s]

[{'nsites': 2, 'volume': 43.625325465502776, 'density': 2.224545043829438}, {'nsites': 2, 'volume': 43.625325465502776, 'density': 2.224545043829438}, {'nsites': 2, 'volume': 46.09614833243692, 'density': 2.1053060843576104}]Thought: The response contains multiple entries for stable NaCl with the same formula but different volumes and densities. To provide a concise answer, I will calculate the average volume per atom by dividing the volume by the number of sites (nsites) for each entry and then provide the average density.

Action:
```
{
  "action": "Final Answer",
  "action_input": "The stable NaCl has an average mass density of approximately 2.17 g/cm^3 and an average volume per atom of approximately 22.58 Å^3."
}
```Thought: The response contains multiple entries for stable NaCl with the same formula but different volumes and densities. To provide a concise answer, I will calculate the average volume per atom by dividing the volume by the number of sites (nsites) for each entry and

{'input': 'Please give me the mass density (in g/cm^3) and volume per atom (in Angstrom^3/atom) of stable NaCl.',
 'chat_history': [],
 'output': 'The stable NaCl has an average mass density of approximately 2.17 g/cm^3 and an average volume per atom of approximately 22.58 Å^3/atom.'}

In [4]:

llm_gpt = ChatOpenAI(
    temperature=0.7,
    model="gpt-3.5-turbo-1106",
    openai_api_key=OPENAI_API_KEY,
    openai_organization=OPENAI_ORGANIZATION,
    # streaming=True
)

llm_gpt.invoke(agent_executor.memory.chat_memory.messages[0].content)


AIMessage(content='The mass density of stable NaCl is 2.165 g/cm^3, and the volume per atom is 37.2 Å^3/atom.', response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 38, 'total_tokens': 71}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_89448ee5dc', 'finish_reason': 'stop', 'logprobs': None})